# Data Analysis

Ryry :)

### Data analysis
1. Look at columns, graph out variance of columns (min/max) .describe
    * Document output of .describe on each column
#### -> 2. Find which column is a finite value vs. continuous values — get to know data better
 - encode subcategories of categorical columns
 - barchart of frequency when given hashed value appears
 
2. Scatter plot to find outliers
- hypothesize about what I can feed a model 
3. Find best application of random forest on raw data 
4. Then run a neural net on cleaned data 


### Experiment with different models on cleaned data
1. Start small — try one model with one feature to predict click
    * Run a NN on one feature to predict click
    * Then try another feature, and so on
2. Try 10 different models
    * SGD descent classifier — look up different tensorflow models, etc. 
    * Document the value of each model
        * Might not find perfect model but we will know. Make a graph (the width of the network vs. depth network vs. loss of network) 
        * Describe a line through time that shows how tweaking different hyper parameters effects the model —doing data discovery by letting the computer do data discovery for us, while we tweak the hyper parameter)
        * Document everything — ask questions like:
            * When does the model start overfitting?
            * What happens if my neural net is 100? 10 neurons wide? compare
            * Try five layers deep vs. One
            * In each comparison, was there any value? Is there a point of diminishing return?
            * Try different GPUs, etc.

### Informed feature engineering (when we know something about the ad-space): 
(avoid feature engineering unless we have domain knowledge of the features — let the models do the work, emphasize normalizing/standardizing data and controlling/understanding column variance to get better model performance — before feature engineering)

We can feature engineer things like banners, we see ad banners every day, try this:
1. Use K-Means on ads — find natural divisions in banners 
2. Try to find clusters of information around ads
3. Make classes for clusters of ads (i.e., square ads, long rectangular banners, etc.)
    * Make scatterplots of the heights, widths of ad banners/size
    * Turn those into one column — make ordinal ’types’ of banners

Subdivide by website — see if we can be more predictive based on the destination site_id, etc.
* Encode unique groups of hashed values of site_id column (one hot encoder)
* Then test, we can discover that maybe “site_a” provides a higher accuracy and so on. 

*Don’t need recurrent neural net or a convolutional neural net — Josh recommends a basic neural net

### References
 - CTR: https://towardsdatascience.com/mobile-ads-click-through-rate-ctr-prediction-44fdac40c6ff
 - the sage wisdom of AMLI instructors & TAs

## Import Libraries and Data

In [2]:
import pandas as pd
import numpy as np
import multiprocessing as mp
import psutil
import random
import datetime as datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
# read in the Avazu - criteo labs - csv file
# rand_sample_csv is a randomized subset (1% the size) of the sample_csv which is ~400k instances 

df = pd.read_csv('rand_sample_eng.csv')


# Data Exploration

In [4]:
df.head(5)

,Unnamed: 0,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,...,C15,C16,C17,C18,C19,C20,C21,new_date,new_time,day_of_week
0,0,10004510652136496837,0,14102100,1005,0,543a539e,c7ca3108,3e814130,ecad2386,...,320,50,2333,0,39,-1,157,2014-10-21,00:00:00,1
1,1,10007164336863914220,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,320,50,1722,0,35,-1,79,2014-10-21,00:00:00,1
2,2,10076859283156800622,0,14102100,1002,0,f17ebd97,c4e18dd6,50e219e0,ecad2386,...,216,36,2497,3,43,100151,42,2014-10-21,00:00:00,1
3,3,10078825124049580646,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,320,50,1722,0,35,-1,79,2014-10-21,00:00:00,1
4,4,10085233430943183912,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,320,50,1722,0,35,-1,79,2014-10-21,00:00:00,1


In [5]:
df.describe()

,Unnamed: 0,id,click,hour,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,day_of_week
count,404410.000000,4.044100e+05,404410.000000,4.044100e+05,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000
mean,202204.500000,9.213896e+18,0.169991,1.410256e+07,1004.967372,0.288638,1.014695,0.331688,18844.936193,318.795856,60.076581,2112.733881,1.431426,227.891140,53239.462209,83.304955,2.601259
std,116743.255518,5.319411e+18,0.375625,2.968127e+02,1.090207,0.504033,0.523959,0.855877,4947.526554,20.667022,47.023691,607.929983,1.325359,351.686105,49955.391770,70.251537,1.727362
min,0.000000,7.306813e+13,0.000000,1.410210e+07,1001.000000,0.000000,0.000000,0.000000,375.000000,120.000000,20.000000,112.000000,0.000000,33.000000,-1.000000,1.000000,0.000000
25%,101102.250000,4.607240e+18,0.000000,1.410230e+07,1005.000000,0.000000,1.000000,0.000000,16920.000000,320.000000,50.000000,1863.000000,0.000000,35.000000,-1.000000,23.000000,1.000000
50%,202204.500000,9.218443e+18,0.000000,1.410260e+07,1005.000000,0.000000,1.000000,0.000000,20346.000000,320.000000,50.000000,2323.000000,2.000000,39.000000,100048.000000,61.000000,2.000000
75%,303306.750000,1.382200e+19,0.000000,1.410281e+07,1005.000000,1.000000,1.000000,0.000000,21893.000000,320.000000,50.000000,2526.000000,3.000000,171.000000,100086.000000,101.000000,4.000000
max,404409.000000,1.844673e+19,1.000000,1.410302e+07,1012.000000,7.000000,5.000000,5.000000,24043.000000,1024.000000,1024.000000,2757.000000,3.000000,1839.000000,100248.000000,255.000000,6.000000


In [6]:
df.shape # this sample has 404,410 rows of data with 26 columns

(404410, 28)

In [7]:
df.dtypes
# avazu: "all integer features are categorical variables, all IDs, no numerical meaning"

Unnamed: 0           int64
id                  uint64
click                int64
hour                 int64
C1                   int64
banner_pos           int64
site_id             object
site_domain         object
site_category       object
app_id              object
app_domain          object
app_category        object
device_id           object
device_ip           object
device_model        object
device_type          int64
device_conn_type     int64
C14                  int64
C15                  int64
C16                  int64
C17                  int64
C18                  int64
C19                  int64
C20                  int64
C21                  int64
new_date            object
new_time            object
day_of_week          int64
dtype: object

In [8]:
# what do the columns mean?

 - id: ad identifier
 - click: 0/1 for non-click/click
 - hour: format is YYMMDDHH
 - C1 — anonymized categorical variable
 - banner_pos
 - site_id
 - site_domain
 - site_category
 - app_id
 - app_domain
 - app_category
 - device_id
 - device_ip
 - device_model
 - device_type
 - device_conn_type
 - C14-C21 — anonymized categorical variables

# Data Preprocessing

In [9]:
#  unnamed column are columns that are created when a dataframe is converted to a csv. 
# 'Unnamed: 0', 'Unnamed: 0.1' are row indexes which were tansposed into columns.
df_new = df.drop(['Unnamed: 0'], axis=1) 

In [10]:
df_new.shape

(404410, 27)

In [11]:
# check for missing values

df_new.isnull().sum()


id                  0
click               0
hour                0
C1                  0
banner_pos          0
site_id             0
site_domain         0
site_category       0
app_id              0
app_domain          0
app_category        0
device_id           0
device_ip           0
device_model        0
device_type         0
device_conn_type    0
C14                 0
C15                 0
C16                 0
C17                 0
C18                 0
C19                 0
C20                 0
C21                 0
new_date            0
new_time            0
day_of_week         0
dtype: int64

In [12]:
# summed list of each column for df_new, looking for inconsistencies

for col in df_new.columns.values:
    total = len(df_new[col].unique())
    print(str(col) + " " + "total: " + str(total))

id total: 404410
click total: 2
hour total: 240
C1 total: 7
banner_pos total: 7
site_id total: 2195
site_domain total: 2172
site_category total: 21
app_id total: 2305
app_domain total: 153
app_category total: 26
device_id total: 64913
device_ip total: 262453
device_model total: 4369
device_type total: 5
device_conn_type total: 4
C14 total: 2067
C15 total: 8
C16 total: 9
C17 total: 415
C18 total: 4
C19 total: 65
C20 total: 159
C21 total: 60
new_date total: 10
new_time total: 24
day_of_week total: 7


Each instance of a column are hashed values of an original ID.
Hashing was done to anonymize the services contributing ad data to this dataset.
For illustrative/descriptive purposes we will treat each hashed value as names or in other fictional contexts (e.g.,'7801e8d9' = 'www.overstock.com'). (Thank you Naomi!)

# Feature Engineering
## Hour & Date 

In [13]:
# check hour column data type
df_new.hour.dtype

dtype('int64')

In [14]:
# separate the date and time
parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')
df_new['new_hour'] = df_new.hour.astype(str).apply(parse_date)
df_new['new_hour']

0        2014-10-21 00:00:00
1        2014-10-21 00:00:00
2        2014-10-21 00:00:00
3        2014-10-21 00:00:00
4        2014-10-21 00:00:00
5        2014-10-21 00:00:00
6        2014-10-21 00:00:00
7        2014-10-21 00:00:00
8        2014-10-21 00:00:00
9        2014-10-21 00:00:00
10       2014-10-21 00:00:00
11       2014-10-21 00:00:00
12       2014-10-21 00:00:00
13       2014-10-21 00:00:00
14       2014-10-21 00:00:00
15       2014-10-21 00:00:00
16       2014-10-21 00:00:00
17       2014-10-21 00:00:00
18       2014-10-21 00:00:00
19       2014-10-21 00:00:00
20       2014-10-21 00:00:00
21       2014-10-21 00:00:00
22       2014-10-21 00:00:00
23       2014-10-21 00:00:00
24       2014-10-21 00:00:00
25       2014-10-21 00:00:00
26       2014-10-21 00:00:00
27       2014-10-21 00:00:00
28       2014-10-21 00:00:00
29       2014-10-21 00:00:00
                 ...        
404380   2014-10-30 23:00:00
404381   2014-10-30 23:00:00
404382   2014-10-30 23:00:00
404383   2014-

In [15]:
# check if column 'new_hour' was created and parsed to string
df_new.head(3)

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C16,C17,C18,C19,C20,C21,new_date,new_time,day_of_week,new_hour
0,10004510652136496837,0,14102100,1005,0,543a539e,c7ca3108,3e814130,ecad2386,7801e8d9,...,50,2333,0,39,-1,157,2014-10-21,00:00:00,1,2014-10-21
1,10007164336863914220,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,50,1722,0,35,-1,79,2014-10-21,00:00:00,1,2014-10-21
2,10076859283156800622,0,14102100,1002,0,f17ebd97,c4e18dd6,50e219e0,ecad2386,7801e8d9,...,36,2497,3,43,100151,42,2014-10-21,00:00:00,1,2014-10-21


In [16]:
#confirm dtype of new_hour
df_new.new_hour.dtype

dtype('<M8[ns]')

In [17]:
# create new_date & new_time columns from parsed new_hour column
df_new['date'] = [d.date() for d in df_new['new_hour']]
df_new['time'] = [d.time() for d in df_new['new_hour']]

In [18]:
#check if columns were established properly
df_new.head(3)

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C18,C19,C20,C21,new_date,new_time,day_of_week,new_hour,date,time
0,10004510652136496837,0,14102100,1005,0,543a539e,c7ca3108,3e814130,ecad2386,7801e8d9,...,0,39,-1,157,2014-10-21,00:00:00,1,2014-10-21,2014-10-21,00:00:00
1,10007164336863914220,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,0,35,-1,79,2014-10-21,00:00:00,1,2014-10-21,2014-10-21,00:00:00
2,10076859283156800622,0,14102100,1002,0,f17ebd97,c4e18dd6,50e219e0,ecad2386,7801e8d9,...,3,43,100151,42,2014-10-21,00:00:00,1,2014-10-21,2014-10-21,00:00:00


In [19]:
df_new.dtypes

id                          uint64
click                        int64
hour                         int64
C1                           int64
banner_pos                   int64
site_id                     object
site_domain                 object
site_category               object
app_id                      object
app_domain                  object
app_category                object
device_id                   object
device_ip                   object
device_model                object
device_type                  int64
device_conn_type             int64
C14                          int64
C15                          int64
C16                          int64
C17                          int64
C18                          int64
C19                          int64
C20                          int64
C21                          int64
new_date                    object
new_time                    object
day_of_week                  int64
new_hour            datetime64[ns]
date                

In [20]:
# drop redundant cols
df_tmp = df_new.drop(['new_hour', 'hour'], axis=1)

In [21]:
df_tmp.head(2)

,id,click,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,C17,C18,C19,C20,C21,new_date,new_time,day_of_week,date,time
0,10004510652136496837,0,1005,0,543a539e,c7ca3108,3e814130,ecad2386,7801e8d9,07d7df22,...,2333,0,39,-1,157,2014-10-21,00:00:00,1,2014-10-21,00:00:00
1,10007164336863914220,1,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,1722,0,35,-1,79,2014-10-21,00:00:00,1,2014-10-21,00:00:00


In [22]:
# sannity check of summed columns for unique vals

for col in df_tmp.columns.values:
    total = len(df_tmp[col].unique())
    val = df_tmp[col].unique()
    print(str(col) + " " + "total: " + str(total))

id total: 404410
click total: 2
C1 total: 7
banner_pos total: 7
site_id total: 2195
site_domain total: 2172
site_category total: 21
app_id total: 2305
app_domain total: 153
app_category total: 26
device_id total: 64913
device_ip total: 262453
device_model total: 4369
device_type total: 5
device_conn_type total: 4
C14 total: 2067
C15 total: 8
C16 total: 9
C17 total: 415
C18 total: 4
C19 total: 65
C20 total: 159
C21 total: 60
new_date total: 10
new_time total: 24
day_of_week total: 7
date total: 10
time total: 24


# Feature engineering cont. 
### device_type converted into binary columns by device_type 


In [23]:
# iterate through columns and print the unique values of each column
for col in df_tmp.columns.values:
    val = df_tmp[col].unique()
    print(str(col) + " " + ", val: " + str(val))

id , val: [10004510652136496837 10007164336863914220 10076859283156800622 ...
  9930625418032326788  9953588061726377330  9959058523366506236]
click , val: [0 1]
C1 , val: [1005 1002 1010 1007 1008 1012 1001]
banner_pos , val: [0 1 2 5 7 4 3]
site_id , val: ['543a539e' '1fbe01fe' 'f17ebd97' ... '9fd919ea' '1b72ccd8' '5a51436e']
site_domain , val: ['c7ca3108' 'f3845767' 'c4e18dd6' ... '0da06afc' '3e87e1c9' '645c06d3']
site_category , val: ['3e814130' '28905ebd' '50e219e0' '76b2941d' 'f028772b' 'f66779e6'
 '0569f928' '335d28a8' '72722551' '75fa27f6' 'c0dd3be3' 'a818d37a'
 '8fd0aea4' '70fb0e29' 'dedf689d' 'e787de0e' '5378d028' 'bcf865d9'
 '42a36e14' '9ccfa2ea' 'c706e647']
app_id , val: ['ecad2386' '1779deee' 'febd1138' ... '96f19b66' '5717fe5d' '404b2054']
app_domain , val: ['7801e8d9' '2347f47a' '82e27996' '45a51db4' '5c5a694b' 'afdf1f54'
 'aefc06bd' 'ae637522' 'd9b5648e' '828da833' '5b9c592b' '0654b444'
 '885c7f3f' 'b8d325c3' 'b5f3b24a' 'ad63ec9b' '33da2e74' '43cf4f06'
 '15ec7f39' '18eb

In [24]:
# use device_type as practice. There are 5 unique vals -- smaller number is easier to work with
df_tmp.device_type.nunique()

5

In [25]:
# store df_tmp.device_type as var for ease of re-use
dvc_type = df_tmp.device_type

In [26]:
# check instance of dvc_type
dvc_type[0]

1

In [27]:
# val counts gives me the count of each unique values 
dvc_type.value_counts()

1    373412
0     22074
4      7676
5      1247
2         1
Name: device_type, dtype: int64

In [28]:
# make var to hold col 'names' based off unique values stored as a list
col_names = df_tmp['device_type'].unique().tolist()
col_names

[1, 0, 4, 5, 2]

In [29]:
# check it
df_tmp.head(3)

,id,click,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,C17,C18,C19,C20,C21,new_date,new_time,day_of_week,date,time
0,10004510652136496837,0,1005,0,543a539e,c7ca3108,3e814130,ecad2386,7801e8d9,07d7df22,...,2333,0,39,-1,157,2014-10-21,00:00:00,1,2014-10-21,00:00:00
1,10007164336863914220,1,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,1722,0,35,-1,79,2014-10-21,00:00:00,1,2014-10-21,00:00:00
2,10076859283156800622,0,1002,0,f17ebd97,c4e18dd6,50e219e0,ecad2386,7801e8d9,07d7df22,...,2497,3,43,100151,42,2014-10-21,00:00:00,1,2014-10-21,00:00:00


In [30]:
df_tmp[col_names] = pd.get_dummies(df_tmp['device_type'])
df_tmp[col_names].describe()

,1,0,4,5,2
count,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000
mean,0.054583,0.923350,0.000002,0.018981,0.003084
std,0.227165,0.266036,0.001572,0.136457,0.055444
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
# 2 unique numbers for column 1, this means it is either 1 or not 1?
df_tmp[col_names][1].nunique()

2

In [32]:
# check for missing values -- there are none, good.

df_tmp.isnull().sum()

id                  0
click               0
C1                  0
banner_pos          0
site_id             0
site_domain         0
site_category       0
app_id              0
app_domain          0
app_category        0
device_id           0
device_ip           0
device_model        0
device_type         0
device_conn_type    0
C14                 0
C15                 0
C16                 0
C17                 0
C18                 0
C19                 0
C20                 0
C21                 0
new_date            0
new_time            0
day_of_week         0
date                0
time                0
1                   0
0                   0
4                   0
5                   0
2                   0
dtype: int64

In [33]:
# ya final rows are columns stratified by device type
df_tmp.shape

(404410, 33)

In [34]:
# confirm successful selection of last 5 cols -- device types
df_tmp[df_tmp.columns[-5:]].head(3)

,1,0,4,5,2
0,0,1,0,0,0
1,0,1,0,0,0
2,1,0,0,0,0


In [35]:
# rename device_type col name for readability / understanding
df_dvtype = df_tmp
#df_dvtype.columns = ['a', 'b']
df_dvtype.head(3)

,id,click,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,new_date,new_time,day_of_week,date,time,1,0,4,5,2
0,10004510652136496837,0,1005,0,543a539e,c7ca3108,3e814130,ecad2386,7801e8d9,07d7df22,...,2014-10-21,00:00:00,1,2014-10-21,00:00:00,0,1,0,0,0
1,10007164336863914220,1,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,2014-10-21,00:00:00,1,2014-10-21,00:00:00,0,1,0,0,0
2,10076859283156800622,0,1002,0,f17ebd97,c4e18dd6,50e219e0,ecad2386,7801e8d9,07d7df22,...,2014-10-21,00:00:00,1,2014-10-21,00:00:00,1,0,0,0,0


In [36]:
df_dvtype = df_dvtype.rename({1:'device_type_a',0:'device_type_b',4:'device_type_c',5:'device_type_d', 2:'device_type_e'}, axis='columns')
df_dvtype.columns

Index(['id', 'click', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'new_date', 'new_time',
       'day_of_week', 'date', 'time', 'device_type_a', 'device_type_b',
       'device_type_c', 'device_type_d', 'device_type_e'],
      dtype='object')

In [37]:
df_num = df_dvtype.drop(columns = ['id', 'new_time', 'new_date'])

In [38]:
df_num.columns

Index(['click', 'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category',
       'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip',
       'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16',
       'C17', 'C18', 'C19', 'C20', 'C21', 'day_of_week', 'date', 'time',
       'device_type_a', 'device_type_b', 'device_type_c', 'device_type_d',
       'device_type_e'],
      dtype='object')

In [39]:
df_num.dtypes

click                int64
C1                   int64
banner_pos           int64
site_id             object
site_domain         object
site_category       object
app_id              object
app_domain          object
app_category        object
device_id           object
device_ip           object
device_model        object
device_type          int64
device_conn_type     int64
C14                  int64
C15                  int64
C16                  int64
C17                  int64
C18                  int64
C19                  int64
C20                  int64
C21                  int64
day_of_week          int64
date                object
time                object
device_type_a        uint8
device_type_b        uint8
device_type_c        uint8
device_type_d        uint8
device_type_e        uint8
dtype: object

In [40]:
# converting non-numerical columns into numerical columns
categories = ['site_id', 'site_domain','site_category', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model', 'date', 'time']

for category in categories:
  df_num[category] = df_num[category].astype('category').cat.codes

In [41]:
# check dtypes
df_num.dtypes

click               int64
C1                  int64
banner_pos          int64
site_id             int16
site_domain         int16
site_category        int8
app_id              int16
app_domain          int16
app_category         int8
device_id           int32
device_ip           int32
device_model        int16
device_type         int64
device_conn_type    int64
C14                 int64
C15                 int64
C16                 int64
C17                 int64
C18                 int64
C19                 int64
C20                 int64
C21                 int64
day_of_week         int64
date                 int8
time                 int8
device_type_a       uint8
device_type_b       uint8
device_type_c       uint8
device_type_d       uint8
device_type_e       uint8
dtype: object

In [42]:
df_num.head(3)
df_num.tail(3)

,click,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,...,C20,C21,day_of_week,date,time,device_type_a,device_type_b,device_type_c,device_type_d,device_type_e
404407,0,1005,0,263,2056,1,2129,65,0,43137,...,-1,51,3,9,23,0,1,0,0,0
404408,0,1005,0,1145,1680,5,1396,17,24,43137,...,-1,221,3,9,23,0,1,0,0,0
404409,0,1005,0,263,2056,1,2129,65,0,43137,...,-1,51,3,9,23,0,1,0,0,0


In [43]:
df_num.device_id.nunique()

64913

In [44]:
df_tmp.device_id.nunique()

64913

In [56]:
# just checking dow -- day_of_week is good, date is numbered 0-9 as there are 10 days of data
df_num.day_of_week.describe()

count    404410.000000
mean          2.601259
std           1.727362
min           0.000000
25%           1.000000
50%           2.000000
75%           4.000000
max           6.000000
Name: day_of_week, dtype: float64

# Original features

 - Target feature : click
 - Site features : site_id, site_domain, site_category
 - App feature: app_id, app_domain, app_category
 - Device feature: device_id, device_ip, device_model, device_type, device_conn_type
 - Anonymized categorical features: C14-C21

# New Features

### All features are numerical (df_num)
- Target feature : click
 - Site features : site_id, site_domain, site_category
 - App feature: app_id, app_domain, app_category
 - Device feature: device_id[a,b,c,d,e], device_ip, device_model, device_type, device_conn_type
 - Anonymized numerical features: C14-C21

# Data Analysis


In [ ]:
df_da = df_num

In [48]:
# encode these categories
for col in df_num.columns.values:
    total = len(df_num[col].unique())
    val = df_num[col].unique()
    if total < 26 and total > 2:
        print(str(col) + " " + "total: " + str(total))

C1 total: 7
banner_pos total: 7
site_category total: 21
device_type total: 5
device_conn_type total: 4
C15 total: 8
C16 total: 9
C18 total: 4
day_of_week total: 7
date total: 10
time total: 24


In [54]:
df_num.C1.dtype

dtype('int64')

In [55]:
# try onehotencoder 
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')

X = df_num.C1


In [ ]:
#enc.fit(X)

#OneHotEncoder(categorical_features=None, categories=None,dtype='numpy.int64', handle_unknown='ignore', n_values=None, sparse=True)

#enc.categories_[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]

#enc.transform([['Female', 1], ['Male', 4]]).toarray()

#array([[1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])

#enc.inverse_transform([[0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])

#array([['Male', 1],[None, 2]], dtype=object)

#enc.get_feature_names()

#array(['x0_Female', 'x0_Male', 'x1_1', 'x1_2', 'x1_3'], dtype=object)

# Standardize / Normalize Data

In [ ]:
# describe each column
for col in df_num.columns:
    mean = df_num[col].mean()
    print(str(col) + " " + "stats are: " + str(mean))

In [ ]:
# correlations
df_da.corr()

In [ ]:
# filter out outliers
#df_normal = df_num[np.abs(df_num.Data-df_num.Data.mean()) <= (3*df_num.Data.std())]
# keep only the ones that are within +3 to -3 standard deviations in the column 'Data'.

# keep anything outside of +3 or -3 stdvs
#df_outliers = df_num[~(np.abs(df_num.Data-df_num.Data.mean()) > (3*df_num.Data.std()))]


# Random Forest
## Classification using sklearn
### baseline metrics

In [ ]:
# baseline random forest
features = ['C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'date', 'time', 'device_type_a', 'device_type_b', 'device_type_c', 'device_type_d', 'device_type_e']
target = ['click']

X_train, X_test, y_train, y_test = train_test_split(df_num[features], df_num[target], test_size = 0.2, random_state = 0)

In [ ]:
y_train.head(3)

In [ ]:
X_train.head(3)

In [ ]:
#running random forest algorithm

from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=1000, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test)  



In [ ]:
Y_pred.shape